# Projecting Food Insecurity Rates in the US by County
## Feature Engineering
The following process imports a cleaned dataset produced from [cleaning_pt2.ipynb.](notebooks/cleaning_pt2) This notebook is used to produce new features that will be used in the modeling process.
### Flatiron School Data Science Capstone<br>By Khyatee Desai

In [781]:
import pandas as pd
import numpy as np
from itertools import combinations
import sklearn
from sklearn.linear_model import LinearRegression
pd.set_option('display.max_columns', None)
import pickle
import warnings
warnings.filterwarnings('ignore')

In [782]:
with open('../pickled/fully_cleaned_data.pickle', "rb") as input_file:
    df = pickle.load(input_file) 
df

,FIPS,Rent,Year,coc_number,Houseless_rate,Sheltered_rate,Unsheltered_rate,hh_med_income,pop_disabled,pop_hs_grad,pop_bachelors,pop_grad_degree,pop_priv_health,pop_public_health,pop_no_health,pop_total,percent_hh_poverty,hh_avg_size,pop_65+,hh_no_vehicle,num_hh,pop_non_citizen,hh_SNAP,State,County,TOT_POP,TOT_MALE,TOT_FEMALE,TOT_WHITE,TOT_BLACK,TOT_NATIVE,TOT_ASIAN,TOT_PACIFIC,TOT_LATINX,State/County,FI Rate,Low Threshold Type,High Threshold Type,Cost Per Meal,Num_wholesale,Num_restaraunts,Num_grocery,Total_workforce,Employed,Unemployed,Unemployment_rate
0,01073,1019.995960,2014,AL-500,0.001396,0.000961,0.000435,45239.0,94584.0,117854.0,81626.0,52774.0,431638.0,211570.0,81336.0,658834.0,14.8,2.48,87036.0,93630.0,259397.0,17519.0,39967.0,Alabama,Jefferson County,659972.0,312491.0,347481.0,355635.0,284082.0,2029.0,10378.0,347.0,24099.0,"Jefferson County, AL",0.197,SNAP,Other Nutrition Program,2.93,483.0,2693.0,400.0,312131.0,292505.0,19626.0,6.3
1,01117,1229.755051,2014,AL-500,0.001396,0.000961,0.000435,69723.0,22792.0,28911.0,35773.0,18511.0,159655.0,42429.0,19175.0,201168.0,6.2,2.65,23404.0,19762.0,74790.0,7624.0,4706.0,Alabama,Shelby County,206280.0,100304.0,105976.0,174094.0,24247.0,805.0,4403.0,101.0,11872.0,"Shelby County, AL",0.105,SNAP,Other Nutrition Program,3.37,1.0,743.0,2706.0,107208.0,102400.0,4808.0,4.5
2,04003,1051.250000,2014,AZ-500,0.001469,0.000826,0.000643,45974.0,20341.0,21109.0,12968.0,7566.0,76099.0,50498.0,14868.0,130807.0,13.1,2.47,23593.0,16328.0,48846.0,7947.0,7812.0,Arizona,Cochise County,127314.0,64661.0,62653.0,112238.0,5737.0,2165.0,2757.0,510.0,44374.0,"Cochise County, AZ",0.161,"SNAP, Other Nutrition Programs","SNAP, Other Nutrition Programs",2.81,800.0,72.0,340.0,50969.0,46682.0,4287.0,8.4
3,04013,1095.670228,2014,AZ-502,0.001318,0.001084,0.000235,53689.0,399455.0,593094.0,490927.0,273108.0,2444443.0,1204681.0,646167.0,3947382.0,12.7,2.74,507428.0,546028.0,1424244.0,373532.0,171581.0,Arizona,Maricopa County,4093648.0,2024659.0,2068989.0,3449404.0,235660.0,112383.0,172425.0,11190.0,1239835.0,"Maricopa County, AZ",0.158,"SNAP, Other Nutrition Programs","SNAP, Other Nutrition Programs",2.90,2389.0,16857.0,6320.0,1961997.0,1848119.0,113878.0,5.8
4,04019,928.546429,2014,AZ-501,0.002170,0.001705,0.000465,46233.0,133694.0,149147.0,115392.0,81406.0,592298.0,364938.0,141211.0,993144.0,13.2,2.50,162075.0,149710.0,386155.0,69636.0,57099.0,Arizona,Pima County,1004229.0,494684.0,509545.0,858334.0,41043.0,42683.0,31905.0,2266.0,363063.0,"Pima County, AZ",0.154,"SNAP, Other Nutrition Programs","SNAP, Other Nutrition Programs",2.85,3591.0,838.0,1604.0,463126.0,435183.0,27943.0,6.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3559,49999,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3560,51999,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3561,53999,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3562,54999,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


### Create a percentage for each demographic

In [783]:
df.replace(-888888888.0,np.nan,inplace=True)

In [784]:
df['Percent_male'] = df['TOT_MALE']/df['TOT_POP']
df['Percent_female'] = df['TOT_FEMALE']/df['TOT_POP']
df['Percent_white'] = df['TOT_WHITE']/df['TOT_POP']
df['Percent_Black'] = df['TOT_BLACK']/df['TOT_POP']
df['Percent_native'] = df['TOT_NATIVE']/df['TOT_POP']
df['Percent_asian'] = df['TOT_ASIAN']/df['TOT_POP']
df['Percent_pacific'] = df['TOT_PACIFIC']/df['TOT_POP']
df['Percent_latinX'] = df['TOT_LATINX']/df['TOT_POP']

### Percentage PoC
Percentage of a county that is not white

In [785]:
df['Percent_PoC'] = 1-df['Percent_white']

### Workforce as a percentage of total population
Workforce represented as percentage, so that it can be compared across different counties

In [786]:
# percent of population that is working
df['Percent_working'] = df['Total_workforce']/df['TOT_POP']


### Sum of Food Establishments
Total number of food retail businesses, which is the sum of the three categories (wholesalers, restaraunts, and grocery stores)

In [787]:
df['Total_food_retail'] = df['Num_wholesale'].fillna(0)+ df['Num_restaraunts'].fillna(0)+df['Num_grocery'].fillna(0)

### Population divided by number of food establishments 
Looking at prevalence of food establishments as a function of population - aka how many Food Retail establishments exist per person within a County

In [788]:
df['Food_retail_per_person'] = df['Total_food_retail']/df['TOT_POP']


## ACS Survey Percentages
Take percentages of all features taken from the ACS survey, which use a slightly different (~5%) total population number for the denominator

In [789]:
df['Percent_disabled'] = df['pop_disabled']/df['TOT_POP']
df['Percent_hs_grad'] = df['pop_hs_grad']/df['TOT_POP']
df['Percent_bachelors'] = df['pop_bachelors']/df['TOT_POP']
df['Percent_grad_degree'] = df['pop_grad_degree']/df['TOT_POP']
df['Percent_priv_health'] = df['pop_priv_health']/df['TOT_POP']
df['Percent_public_health'] = df['pop_public_health']/df['TOT_POP']
df['Percent_no_health'] = df['pop_no_health']/df['TOT_POP']
df['Percent_65+'] = df['pop_65+']/df['TOT_POP']
df['Percent_non_citizen'] = df['pop_non_citizen']/df['TOT_POP']
df['Percent_hh_no_vehicle'] = df['hh_no_vehicle']/df['num_hh']
df['Percent_hh_SNAP'] = df['hh_SNAP']/df['num_hh']

### Drop raw count columns after deriving percentages

In [790]:
df.drop(['TOT_MALE','TOT_FEMALE','TOT_WHITE','TOT_BLACK','TOT_NATIVE','TOT_ASIAN','TOT_PACIFIC','TOT_LATINX',
         'pop_disabled', 'pop_hs_grad','pop_bachelors', 'pop_grad_degree', 'pop_priv_health', 'pop_public_health',
        'pop_no_health', 'pop_65+','pop_non_citizen','hh_no_vehicle','hh_SNAP','pop_total'],axis=1,inplace=True)

## Create Percentages for CPS 2020 Data

In [791]:
with open('../pickled/cps_20_data.pickle', "rb") as input_file:
    df_cps_20 = pickle.load(input_file) 

### Education Columns

In [792]:
df_cps_20['Percent_hs_grad'] = df_cps_20['pop_hs_grad'] / df_cps_20['Num_respondants_b']
df_cps_20['Percent_bachelors'] = df_cps_20['pop_bachelors'] / df_cps_20['Num_respondants_b']
df_cps_20['Percent_grad_degree'] = df_cps_20['pop_grad_degree'] / df_cps_20['Num_respondants_b']

### Citizenship Status Columns

In [793]:
df_cps_20['Percent_non_citizen'] = df_cps_20['pop_non_citizen'] / df_cps_20['Num_respondants_b']

### Disability Status Columns

In [794]:
df_cps_20['Percent_disabled'] = df_cps_20['HH_disabled']/ (df_cps_20['HH_disabled']+df_cps_20['HH_not_disabled'] )


### Health Insurance columns

In [795]:
df_cps_20['Percent_no_health'] = (df_cps_20['HH_not_insured'])/(df_cps_20['HH_health_insured']+df_cps_20['HH_not_insured']+df_cps_20['HH_some_insured'])
df_cps_20['Percent_priv_health'] = (df_cps_20['HH_health_priv']+df_cps_20['HH_some_insured_priv']) /(df_cps_20['HH_not_insured_priv'] + df_cps_20['HH_some_insured_priv']+df_cps_20['HH_health_priv'])
df_cps_20['Percent_public_health'] = (df_cps_20['HH_insured_pub']+df_cps_20['HH_some_insured_pub']) /(df_cps_20['HH_no_health_pub'] + df_cps_20['HH_some_insured_pub']+df_cps_20['HH_insured_pub'])


In [796]:
df_cps_20 = df_cps_20.loc[:,['Year','HH_income','HH_size', 'Percent_hs_grad','Percent_bachelors','Percent_grad_degree',
    'Percent_non_citizen','Percent_disabled','Percent_no_health','Percent_priv_health','Percent_public_health']].reset_index()

df_cps_20.rename(columns={'index':'FIPS', 'HH_income':'hh_med_income','HH_size':'hh_avg_size' },inplace=True)

### Concatenate 2020 CPS data with main df

In [797]:
df_no_20 = df[~df.Year.isin(['2020'])]
df_20 = df[df.Year=='2020']

df_20_dropped = df_20.drop(df_cps_20.columns[1:],axis=1)
df_20 = df_cps_20.merge(df_20_dropped, on='FIPS', how='outer')
df_20['Year'] = '2020'
df = pd.concat([df_no_20, df_20])

In [798]:
df

,FIPS,Rent,Year,coc_number,Houseless_rate,Sheltered_rate,Unsheltered_rate,hh_med_income,percent_hh_poverty,hh_avg_size,num_hh,State,County,TOT_POP,State/County,FI Rate,Low Threshold Type,High Threshold Type,Cost Per Meal,Num_wholesale,Num_restaraunts,Num_grocery,Total_workforce,Employed,Unemployed,Unemployment_rate,Percent_male,Percent_female,Percent_white,Percent_Black,Percent_native,Percent_asian,Percent_pacific,Percent_latinX,Percent_PoC,Percent_working,Total_food_retail,Food_retail_per_person,Percent_disabled,Percent_hs_grad,Percent_bachelors,Percent_grad_degree,Percent_priv_health,Percent_public_health,Percent_no_health,Percent_65+,Percent_non_citizen,Percent_hh_no_vehicle,Percent_hh_SNAP
0,01073,1019.995960,2014,AL-500,0.001396,0.000961,0.000435,45239.0,14.8,2.48,259397.0,Alabama,Jefferson County,659972.0,"Jefferson County, AL",0.197,SNAP,Other Nutrition Program,2.93,483.0,2693.0,400.0,312131.000,292505.000,19626.000,6.3000,0.473491,0.526509,0.538864,0.430446,0.003074,0.015725,0.000526,0.036515,0.461136,0.472946,3576.0,0.005418,0.143315,0.178574,0.123681,0.079964,0.654025,0.320574,0.123242,0.131878,0.026545,0.360953,0.154077
1,01117,1229.755051,2014,AL-500,0.001396,0.000961,0.000435,69723.0,6.2,2.65,74790.0,Alabama,Shelby County,206280.0,"Shelby County, AL",0.105,SNAP,Other Nutrition Program,3.37,1.0,743.0,2706.0,107208.000,102400.000,4808.000,4.5000,0.486252,0.513748,0.843969,0.117544,0.003902,0.021345,0.000490,0.057553,0.156031,0.519721,3450.0,0.016725,0.110491,0.140154,0.173420,0.089737,0.773972,0.205686,0.092956,0.113457,0.036959,0.264233,0.062923
2,04003,1051.250000,2014,AZ-500,0.001469,0.000826,0.000643,45974.0,13.1,2.47,48846.0,Arizona,Cochise County,127314.0,"Cochise County, AZ",0.161,"SNAP, Other Nutrition Programs","SNAP, Other Nutrition Programs",2.81,800.0,72.0,340.0,50969.000,46682.000,4287.000,8.4000,0.507886,0.492114,0.881584,0.045062,0.017005,0.021655,0.004006,0.348540,0.118416,0.400341,1212.0,0.009520,0.159770,0.165803,0.101858,0.059428,0.597727,0.396641,0.116782,0.185313,0.062420,0.334275,0.159931
3,04013,1095.670228,2014,AZ-502,0.001318,0.001084,0.000235,53689.0,12.7,2.74,1424244.0,Arizona,Maricopa County,4093648.0,"Maricopa County, AZ",0.158,"SNAP, Other Nutrition Programs","SNAP, Other Nutrition Programs",2.90,2389.0,16857.0,6320.0,1961997.000,1848119.000,113878.000,5.8000,0.494586,0.505414,0.842623,0.057567,0.027453,0.042120,0.002734,0.302868,0.157377,0.479278,25566.0,0.006245,0.097579,0.144882,0.119924,0.066715,0.597131,0.294281,0.157846,0.123955,0.091247,0.383381,0.120472
4,04019,928.546429,2014,AZ-501,0.002170,0.001705,0.000465,46233.0,13.2,2.50,386155.0,Arizona,Pima County,1004229.0,"Pima County, AZ",0.154,"SNAP, Other Nutrition Programs","SNAP, Other Nutrition Programs",2.85,3591.0,838.0,1604.0,463126.000,435183.000,27943.000,6.0000,0.492601,0.507399,0.854719,0.040870,0.042503,0.031771,0.002256,0.361534,0.145281,0.461176,6033.0,0.006008,0.133131,0.148519,0.114906,0.081063,0.589804,0.363401,0.140616,0.161392,0.069343,0.387694,0.147865
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3263,72145,NaN,2020,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"Vega Baja Municipio, PR",NaN,NaN,NaN,NaN,NaN,NaN,NaN,12646.625,11223.375,1423.250,11.2375,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3264,72147,NaN,2020,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"Vieques Municipio, PR",NaN,NaN,NaN,NaN,NaN,NaN,NaN,2476.875,2252.000,224.875,9.0875,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3265,72149,NaN,2020,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"Villalba Municipio, PR",NaN,NaN,NaN,NaN,NaN,NaN,NaN,6819.875,6013.500,806.375,11.6500,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3266,72151,NaN,2020,NaN,NaN,NaN,NaN,NaN,NaN,NaN,

## Impute Missing 2020 Data
2020 Data is only available for 329 Counties, as the CPS surveys are only performed on a subset of the population. The following cells use survey responses to impute missing values for all of the other counties, by taking an average rate of change.

Method: 
- retrieve fips for counties with missing values
- determine average change from 2019-2020 for CPS columns
- apply average rate of change to missing counties

### Determine which Counties have missing 2020 data

In [799]:
missing_fips = df_20['FIPS'][df_20.hh_med_income.isnull()].tolist()
not_missing = df_20['FIPS'][~df_20.hh_med_income.isnull()].tolist()
cps_columns = df_cps_20.columns


### Calculate mean values and percent change for 2019 & 2020

In [800]:
df_19_cps = df[cps_columns][df.Year == '2019']
means_19_20 = df_19_cps[df_19_cps['FIPS'].isin(not_missing)].mean().reset_index().rename(columns={0:'2019 Mean'})

df_20_cps = df[cps_columns][df.Year == '2020']
means_19_20['2020 Mean'] = df_20_cps[df_20_cps['FIPS'].isin(not_missing)].mean().values      
means_19_20[['2019 Mean', '2020 Mean']].pct_change(axis=1)
means_19_20['pct_change'] = (means_19_20['2020 Mean']-means_19_20['2019 Mean'])/means_19_20['2019 Mean']
means_19_20

,index,2019 Mean,2020 Mean,pct_change
0,FIPS,inf,inf,NaN
1,Year,inf,inf,NaN
2,hh_med_income,6.991718e+04,6.872134e+04,-0.017104
3,hh_avg_size,2.648451e+00,1.762491e+00,-0.334520
4,Percent_hs_grad,4.294521e-02,3.861897e-01,7.992614
5,Percent_bachelors,5.738447e-02,1.450319e-01,1.527372
6,Percent_grad_degree,1.373382e-01,8.734611e-02,-0.364007
7,Percent_non_citizen,6.442932e-02,4.038632e-02,-0.373169
8,Percent_disabled,9.741635e-01,2.024197e-02,-0.979221
9,Percent_no_health,7.353092e-02,3.898461e-02,-0.469820


### Apply percent change to generate missing values for 2020
Percent change rate is applied to the 2019 values of counties that are missing values for 2020.

In [801]:
df_19_20 = df_19[df_19.FIPS.isin(missing_fips)].drop('Year',axis=1)
df_19_20['hh_med_income'] = df_19_20['hh_med_income']+ df_19_20['hh_med_income']* -0.017104
df_19_20['hh_avg_size'] = df_19_20['hh_avg_size']+ df_19_20['hh_avg_size']* -0.334520
df_19_20['Percent_hs_grad'] = df_19_20['Percent_hs_grad']+ df_19_20['Percent_hs_grad']* 7.992614
df_19_20['Percent_bachelors'] = df_19_20['Percent_bachelors']+ df_19_20['Percent_bachelors']* 1.527372
df_19_20['Percent_grad_degree'] = df_19_20['Percent_grad_degree']+ df_19_20['Percent_grad_degree']* -0.364007
df_19_20['Percent_non_citizen'] = df_19_20['Percent_non_citizen']+ df_19_20['Percent_non_citizen']* -0.373169
df_19_20['Percent_disabled'] = df_19_20['Percent_disabled']+ df_19_20['Percent_disabled']* -0.979221
df_19_20['Percent_no_health'] = df_19_20['Percent_no_health']+ df_19_20['Percent_no_health']* -0.469820
df_19_20['Percent_priv_health'] = df_19_20['Percent_priv_health']+ df_19_20['Percent_priv_health']* 0.095663
df_19_20['Percent_public_health'] = df_19_20['Percent_public_health']+ df_19_20['Percent_public_health']* 0.403002
df_19_20

,FIPS,hh_med_income,hh_avg_size,Percent_hs_grad,Percent_bachelors,Percent_grad_degree,Percent_non_citizen,Percent_disabled,Percent_no_health,Percent_priv_health,Percent_public_health
0,01001,57726.464976,1.723593,0.539695,0.142091,0.068518,0.008415,0.020289,0.036592,0.761997,0.483212
2,01005,31968.692400,1.657045,1.197752,0.132481,0.035218,0.008506,0.018918,0.054637,0.528880,0.657853
3,01007,46728.841632,1.989785,0.938856,0.134190,0.029621,0.003919,0.019144,0.052109,0.611143,0.525264
4,01009,48513.780768,1.843380,0.679741,0.217921,0.037746,0.010471,0.020560,0.056469,0.736834,0.473094
5,01011,37138.725360,1.830070,1.263293,0.079567,0.033182,0.004468,0.020294,0.058891,0.559926,0.625733
...,...,...,...,...,...,...,...,...,...,...,...
3870,72151,16016.290320,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3871,72153,14490.835728,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3872,72153,14490.835728,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3873,72153,14490.835728,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [802]:
df_no_20 = df[~df.Year.isin(['2020'])]
df_20 = df[df.Year=='2020']

In [803]:
df_20_not_missing_cps = df_20[cps_columns][df_20.FIPS.isin(not_missing)].drop('Year',axis=1)
df_20_cps_all_fips = pd.concat([df_19_20, df_20_not_missing_cps])
df_20_cps_all_fips

,FIPS,hh_med_income,hh_avg_size,Percent_hs_grad,Percent_bachelors,Percent_grad_degree,Percent_non_citizen,Percent_disabled,Percent_no_health,Percent_priv_health,Percent_public_health
0,01001,57726.464976,1.723593,0.539695,0.142091,0.068518,0.008415,0.020289,0.036592,0.761997,0.483212
2,01005,31968.692400,1.657045,1.197752,0.132481,0.035218,0.008506,0.018918,0.054637,0.528880,0.657853
3,01007,46728.841632,1.989785,0.938856,0.134190,0.029621,0.003919,0.019144,0.052109,0.611143,0.525264
4,01009,48513.780768,1.843380,0.679741,0.217921,0.037746,0.010471,0.020560,0.056469,0.736834,0.473094
5,01011,37138.725360,1.830070,1.263293,0.079567,0.033182,0.004468,0.020294,0.058891,0.559926,0.625733
...,...,...,...,...,...,...,...,...,...,...,...
324,55073,47870.090909,1.848485,0.542857,0.171429,0.000000,0.000000,0.000000,0.090909,0.681818,0.545455
325,55101,97732.240000,2.240000,0.512195,0.146341,0.024390,0.024390,0.025641,0.000000,0.846154,0.461538
326,55105,55234.312500,1.875000,0.533333,0.116667,0.083333,0.033333,0.041667,0.000000,0.625000,0.625000
327,55139,75513.866667,1.700000,0.421875,0.203125,0.062500,0.000000,0.000000,0.000000,0.650000,0.550000


In [804]:
not_cps_columns = list(set(df_20.columns)-set(cps_columns[1:]))
df_20_no_cps = df_20[not_cps_columns]
df_20_new = df_20_cps_all_fips.merge(df_20_no_cps,on='FIPS',how='outer')
df_20_new['Year']='2020'
df = pd.concat([df_no_20,df_20_new])

# Impute Missing 2020 Data
Some features, such as demographic information, follow a predictable pattern and are unlikely to be sharply affected by the pandemic, therefore these columns will be imputed using previous yearly data to calculate estimated 2020 values.
### Determine yearly percent change for demographics columns

In [805]:
df.groupby('Year')[['Percent_male','Percent_female','Percent_white','Percent_Black','Percent_native',
        'Percent_asian','Percent_pacific','Percent_latinX','Percent_PoC','Percent_disabled', 'Percent_hs_grad',
     'Percent_bachelors','Percent_grad_degree','Percent_priv_health','Percent_public_health','Percent_no_health']].mean()


,Percent_male,Percent_female,Percent_white,Percent_Black,Percent_native,Percent_asian,Percent_pacific,Percent_latinX,Percent_PoC,Percent_disabled,Percent_hs_grad,Percent_bachelors,Percent_grad_degree,Percent_priv_health,Percent_public_health,Percent_no_health
Year,,,,,,,,,,,,,,,,
2009,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2010,0.499648,0.500352,0.858055,0.090013,0.021626,0.012630,0.000961,0.084257,0.141945,NaN,0.238260,0.084332,0.043167,NaN,NaN,NaN
2011,0.499541,0.500459,0.839700,0.110203,0.018216,0.013882,0.001130,0.064932,0.160300,NaN,0.237480,0.084378,0.043308,NaN,NaN,NaN
2012,0.499773,0.500227,0.838104,0.110548,0.018413,0.014356,0.001165,0.066211,0.161896,0.149516,0.236568,0.085763,0.044233,0.627519,0.331782,0.146020
2013,0.499853,0.500147,0.835338,0.111947,0.018647,0.014926,0.001204,0.067619,0.164662,0.150693,0.237064,0.087005,0.045306,0.619998,0.339705,0.146622
2014,0.500625,0.499375,0.850215,0.092106,0.022659,0.014644,0.001266,0.091117,0.149785,0.150614,0.237156,0.090137,0.047232,0.625153,0.341686,0.140750
2015,0.500818,0.499182,0.848638,0.092428,0.022878,0.015110,0.001301,0.092774,0.151362,0.151874,0.237364,0.091465,0.048343,0.628070,0.350993,0.130266
2016,0.500852,0.499148,0.847006,0.092782,0.023124,0.015556,0.001332,0.094446,0.152994,0.153260,0.236615,0.093128,0.049572,0.631643,0.360194,0.118918
2017,0.500875,0.499125,0.845457,0.093154,0.023354,0.015933,0.001368,0.096121,0.154543,0.153586,0.235818,0.094946,0.050888,0.635518,0.367423,0.107984


In [806]:
percent_change = df.groupby('Year')[['Percent_male','Percent_female','Percent_white','Percent_Black','Percent_native',
        'Percent_asian','Percent_pacific','Percent_latinX','Percent_PoC']].mean().pct_change().mean()
percent_change


Percent_male       0.000192
Percent_female    -0.000192
Percent_white     -0.002104
Percent_Black      0.008908
Percent_native     0.011271
Percent_asian      0.041518
Percent_pacific    0.033430
Percent_latinX     0.028241
Percent_PoC        0.013349
dtype: float64

### Multiply each 2019 value by the calculated percent change by above, to derive a 2020 estimate

In [807]:
dems_19_20 = df[['FIPS','Percent_male','Percent_female','Percent_white','Percent_Black','Percent_native',
        'Percent_asian','Percent_pacific','Percent_latinX','Percent_PoC']][df.Year=='2019'].drop_duplicates()

dems_19_20['Percent_male'] = dems_19_20['Percent_male'] + dems_19_20['Percent_male']* 0.000192
dems_19_20['Percent_female'] = dems_19_20['Percent_female'] + dems_19_20['Percent_female']* -0.000192
dems_19_20['Percent_white'] = dems_19_20['Percent_white']+ dems_19_20['Percent_white']* -0.002104
dems_19_20['Percent_Black'] = dems_19_20['Percent_Black'] +dems_19_20['Percent_Black']* 0.008908
dems_19_20['Percent_native'] = dems_19_20['Percent_native']+ dems_19_20['Percent_native']* 0.011271
dems_19_20['Percent_asian'] = dems_19_20['Percent_asian'] +dems_19_20['Percent_asian']* 0.041518
dems_19_20['Percent_pacific'] = dems_19_20['Percent_pacific'] +dems_19_20['Percent_pacific']* 0.033430
dems_19_20['Percent_latinX'] = dems_19_20['Percent_latinX'] +dems_19_20['Percent_latinX']* 0.028241
dems_19_20['Percent_PoC'] = dems_19_20['Percent_PoC'] + dems_19_20['Percent_PoC']* 0.013349


### Drop nulls, and add the newly calculated 2020 values into the main df

In [808]:
df_20_dropped_null = df[df.Year=='2020'].drop(['Percent_male','Percent_female','Percent_white','Percent_Black',
            'Percent_native','Percent_asian','Percent_pacific','Percent_latinX','Percent_PoC'],axis=1)
df_not_20 = df[df.Year!='2020']
df_20_new = df_20_dropped_null.merge(dems_19_20,on='FIPS',how='outer')

df = pd.concat([df_not_20, df_20_new])

# Polynomial Features
**Note:** Decided to omit polynomial features, because they decreased model performance.

In [809]:
# only using derived percentages, ignore raw counts
continuous_features = ['Rent', 'Houseless_rate','Sheltered_rate', 'Unsheltered_rate', 'TOT_POP',
       'Cost Per Meal', 'Num_wholesale','Num_restaraunts', 'Num_grocery',  'Unemployment_rate', 'Percent_male', 
         'Percent_female','Percent_white', 'Percent_Black', 'Percent_native', 'Percent_asian',
       'Percent_pacific', 'Percent_latinX','Percent_working', 'Total_food_retail','Food_retail_per_person',
        'Percent_disabled','Percent_hs_grad','Percent_bachelors','Percent_grad_degree','Percent_priv_health',
            'Percent_public_health','Percent_no_health','Percent_65+','Percent_non_citizen','Percent_hh_no_vehicle',
                    'Percent_hh_SNAP','percent_hh_poverty','hh_avg_size','num_hh','hh_med_income']


In [810]:
## add squared and cubed polynomials for each continuous feature
# for feat in continuous_features:
#     df[feat+'^2'] = df[feat]**2
#     df[feat+'^3'] = df[feat]**3


# Interaction Features
Create an interaction feature for each combination of continuous features, and add best ones to dataframe

In [811]:
# Generate combinations of features
y = df.dropna()[['FI Rate']]
X = df.dropna()[continuous_features]
interactions = list(combinations(X.columns, 2))
interaction_dict = {}

# run simple regression model with each possible interaction, and save R-squared for each interaction in a dictionary
for interaction in interactions:
    X_copy = X.copy()
    X_copy['interact'] = X_copy[interaction[0]] * X_copy[interaction[1]] 
    X_copy = X_copy.replace([np.inf, -np.inf], 0)
    model = LinearRegression()
    model.fit(X_copy, y)
    interaction_dict[model.score(X_copy, y)] = interaction 


### Add best 50 interactions to dataframe

In [812]:
# Sort the interactions dictionary, and add best 50 interactions to dataframe
top_interactions = sorted(interaction_dict.keys(), reverse = True)[:50]
for interaction in top_interactions:
    feature1 = interaction_dict[interaction][0]
    feature2 = interaction_dict[interaction][1]
    df[feature1+'_X_'+feature2] = df[feature1] * df[feature2] #also add to new_features df


# Log Transformations
Take natural log of each continuous feature, and add these log features to dataframe

In [813]:
for feat in continuous_features:
    df['log_'+feat] = df[feat].map(lambda x: np.log(x))
df = df.replace([np.inf, -np.inf], 0)

# Dummy Variables
High and Low Threshold programs delineate the assistance programs provided by State

In [814]:
# Create dummy variables for high and low threshold programs, and add to dataframe
hi_thresh_dummies = pd.get_dummies(df['High Threshold Type'].astype(str), dtype=int)
hi_thresh_dummies['other'] = hi_thresh_dummies['Other Nutrition Program'] + hi_thresh_dummies['other nutrition pgm']
hi_thresh_dummies.drop(['Other Nutrition Program','other nutrition pgm','nan'],axis=1,inplace=True) # drop last col
hi_thresh_dummies.rename(columns = {'SNAP, Other Nutrition Programs': 'SNAP_other'}, inplace=True)
hi_thresh_dummies.columns = 'Hi_thresh_'+hi_thresh_dummies.columns

low_thresh_dummies = pd.get_dummies(df['Low Threshold Type'].astype(str), dtype=int)
low_thresh_dummies.drop('nan', axis=1,inplace=True)
low_thresh_dummies.rename(columns = {'SNAP, Other Nutrition Programs': 'SNAP_other'}, inplace=True)
low_thresh_dummies.columns = 'Lo_thresh_'+low_thresh_dummies.columns

df = pd.concat([df, low_thresh_dummies, hi_thresh_dummies],axis=1)


### Pickle the new dataframe
Save the dataframe with all new features added (demographic percentages, interactions, logs, and dummies)

In [815]:
with open('../pickled/feature_engineered_data.pickle', "wb") as output_file:
    pickle.dump(df, output_file)